In [1]:
import pickle
from statistics import mean 
import collections
import math
import numpy as np

In [2]:
data = 'adult_income'
method = 'piece'

In [3]:
with open(data+'_'+method+'.pickle', 'rb') as f:
    results = pickle.load(f)

In [4]:
#print(results[0])

In [5]:
sf_query = []
sf_nh_knn = []
sf_nun_knn = []
sf_nun = []
mahalanobis = []
sparsity = []
ood_distance = []
trust_score = []
lipschitz = []

In [6]:
for item in results:
    sf_query.append(item['sf_query'])
    sf_nun.append(item['sf_nun'])
    mahalanobis.append(item['mahalanobis'])
    sparsity.append(item['sparsity'])
    ood_distance.append(item['ood_distance'])
    trust_score.append(item['trust_score'])
    lipschitz.append(item['lipschitz'])

In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# function to scale the values
def scale_values(input_list, feature_range=(0, 1)):
    # Reshape the input list to a 2D array for MinMaxScaler
    values_2d = [[value] for value in input_list]

    # Initialize the MinMaxScaler with the specified feature range
    #scaler = MinMaxScaler(feature_range=feature_range)
    scaler = StandardScaler()

    # Fit and transform the data using the scaler
    scaled_values = scaler.fit_transform(values_2d)

    # Extract the scaled values from the 2D array and return as a list
    scaled_list = [scaled_value[0] for scaled_value in scaled_values]

    return scaled_list

In [8]:
sf_query = [x for xs in sf_query for x in xs]
sf_nun = [x for xs in sf_nun for x in xs]
mahalanobis = [x for xs in mahalanobis for x in xs]
sparsity = [x for xs in sparsity for x in xs]
ood_distance = [x for xs in ood_distance for x in xs]
trust_score = [x for xs in trust_score for x in xs]
trust_score_sc = scale_values(trust_score)
lipschitz = [x for xs in lipschitz for x in xs]
lipschitz_sc = [np.log(x) for x in lipschitz]
#lipschitz_sc = scale_values(lipschitz)

In [12]:
lipschitz_sc

[0.0,
 2.6843391542568114,
 3.302530480260251,
 1.2935105259047357,
 0.099157724093042,
 0.7884918583739541,
 0.47891445364094515,
 0.0,
 1.1912292618082436,
 0.0,
 0.8757231130796268,
 0.23118730524722608,
 2.6987246819345097,
 2.8417774801741476,
 1.2053648146350566,
 2.306604624501878,
 1.4026838374917188,
 0.0,
 3.1747158246762126,
 0.0,
 0.1658021350449484,
 1.460569815159522,
 0.44548507937957327,
 0.15174419170679274,
 0.10347679385587381,
 1.4255735952272308,
 0.9399513833227872,
 0.25738799568876747,
 0.48526539264775476,
 1.4469143705113308,
 0.5265460636249581,
 2.4444352388049477,
 2.1013390961822647,
 0.7503003425062806,
 0.26764196391174666,
 2.2874271313327594,
 0.0,
 0.7232384152756816,
 1.6167191859066203,
 0.017630509319526522,
 0.0,
 0.5063083938113857,
 2.0402517586666202,
 1.4857865930370397,
 2.892770162344772,
 1.7862088289101141,
 1.2048872756014992,
 0.32093166303178533,
 0.9125178790766725,
 0.5003546762813726,
 2.3740545984598524,
 1.6374028940743928,
 1.9837

In [11]:
lipschitz[56]

0.9877960720478952

In [11]:
metric_zip = zip(sf_query, sf_nun, sparsity, ood_distance, trust_score_sc, lipschitz_sc)

In [12]:
unitary = []

# create unitary metric by combining the metrics in a relevant way
for sf_q, sf_n, sp, ood, ts, lips in metric_zip:
    
    if (sp == 0):
        val = sf_q + 0 + ts + (1-ood) + (1-lips)
    else:
        val = sf_q + (1/sp) + ts +(1-ood) + (1-lips)
    
    print(val)
    unitary.append(val)

9.58925159712311
3.216899123089309
3.2176870702344234
1.3286898830515188
2.9321051611648272
2.9267864487052666
2.9960002412055187
3.395799745963362
2.8206426642081066
2.6300379788842907
2.6130578505721207
2.7511473494773195
2.786893728557475
3.051383577007504
2.8956926965290686
1.6442055984766286
3.131264827675513
3.2679186160042804
1.6607923820439743
2.475816271539326
3.3397336012935
2.7955126238790764
2.8067349538907616
2.3322433744027586
2.8412752806446493
2.6431508006068554
2.810276585167714
1.6544634168358394
2.7518640230801794
2.6634289519162744
2.671897427414503
1.2462670671493898
3.1945219613593716
2.93373514992311
3.48098648558056
2.764936678844109
2.629760864313811
2.7627714081872705
3.8563135037423955
3.006229578031497
2.7727149170489334
2.7570357540779282
2.7791411790023344
3.0489104125951685
2.8211130811655982
1.6665740423570936
2.9351959643793433
2.917304591285247
2.1291662460445764
2.637521316311896
2.8516555147881055
2.6593023819330064
3.0086982908665845
3.1241473891083

In [13]:
unitary = mean(unitary)
unitary

2.7988313860026603

In [12]:
sf_query = mean(sf_query)
sf_query

0.12414954712879903

In [33]:
sf_nun = mean(sf_nun)
sf_nun

0.19761579463732462

In [34]:
mahalanobis = mean(mahalanobis)
mahalanobis

4.577524407906172

In [35]:
ood_distance = mean(ood_distance)
ood_distance

0.10831598828772308

In [36]:
# filter trust score
trust_score = [value for value in trust_score if value < 100]
trust_score = mean(trust_score)
trust_score

1.8303869817048086

In [37]:
lipschitz = mean(lipschitz)
lipschitz

27.36931871614887

In [38]:
sp = sparsity.copy()
sp = [1/val if val>0 else 0 for val in sp]
mean(sp)

0.783

In [18]:
sparse = collections.Counter(sparsity)
sparse

Counter({0: 217, 1: 783})

In [19]:
new_dict = {}
three = 0
one = 0
two = 0
for key in sparse:
    if key>=3:
        three += sparse[key]
    elif key == 1:
        one = sparse[key]
    elif key == 2:
        two = sparse[key]

new_dict['1'] = one
new_dict['2'] = two
new_dict['3+'] = three

new_dict

{'1': 783, '2': 0, '3+': 0}

In [20]:
new_dict_p = {}

new_dict_p['1'] = (new_dict['1']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100
new_dict_p['2'] = (new_dict['2']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100
new_dict_p['3+'] = (new_dict['3+']/(new_dict['1']+new_dict['2']+new_dict['3+']))*100

new_dict_p

{'1': 100.0, '2': 0.0, '3+': 0.0}